## Functions necessary for pre-processing



use get_mappings() to see all unique {particles:superstrips} mappings for all training samples (first 100 events of total training data)
<br>

use calc_mappings() to generate all unqiue {particles:superstrips} mappings for all training samples + writes them to file 


In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
from trackml.dataset import load_event, load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event
import timeit
from multiprocessing import Pool
import multiprocessing
import os
from scipy.stats import norm

In [14]:

def load_single_train_event(event_number):
    train_dir = 'data/train_sample/'
    hits, cells, particles, truth = load_event(train_dir+'event00000'+str(event_number))
    return hits, cells, particles, truth


def process_single_event(event_number):
    start = time.time()
    file_name = 'event00000' + str(event_number)
    event_id = file_name
    hits_orig, cells, particles, truth = load_event('data/train_sample/'+event_id)
    merge_by_hit_ids = hits_orig.merge(truth, how = 'inner', on = 'hit_id')
    merge_by_particle_ids = merge_by_hit_ids.merge(particles, how = 'inner', on = 'particle_id')
    partid_dict = {}
    hitloc_dict = {}
    for row in merge_by_particle_ids.itertuples():
        particleID = row.__getattribute__('particle_id')
        volID = row.__getattribute__('volume_id')
        layerID = row.__getattribute__('layer_id') 
        modID = row.__getattribute__('module_id')
        hitID = row.__getattribute__('hit_id')
        key_name = event_id + '-' + str(particleID)
        hitloc_dict = {'hit_id':hitID, 'volume_id':volID, 'layer_id':layerID, 'module_id': modID}
        if key_name in partid_dict:
            partid_dict[key_name].append(hitloc_dict)
        else:
            partid_dict[key_name] = [hitloc_dict]
    
    with open('mappings/'+event_id + '.json','w') as outfile:
        json.dump(partid_dict, outfile)
    
    end = time.time()
    return partid_dict

def calc_mappings():
    start = time.time()
    p = Pool(None) ##uses all available cpu cores, 32 in this case
    p.map(process_single_event, list(range(1000,1100))) #just contents of training-sample for now, i.e first 100 events 
    data_dir = 'mappings'
    all_data = {}
    for folder, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.json'):
                with open(os.path.join(folder,file)) as file:
                    single_event_info = json.load(file)
                    all_data.update(single_event_info)
    with open('training-sample-mappings.json','w') as outfile:
        json.dump(all_data, outfile)
    end = time.time()
    print('elasped time:'+str(end-start))
    return
    
def get_mappings():
    path = 'training-sample-mappings.json'
    with open(path) as file:
        data = json.load(file)
    return data

In [3]:
def reconstruct_test_events ():
    itr = load_dataset ('./data/test_data', parts = ['hits'])
    for event_data in itr:
        event_id = event_data[0]

In [4]:
 #print(timeit.timeit("calc_mappings()",number=1, setup="from __main__ import calc_mappings"))

In [5]:
#can prob be further optimzed 
#dont user toFile param for now
def find_top_hyperstrips(k,toFile=False):
    """
        k: num of hyperstrips we're considering 
        toFile: flag for if we want to write the returned {K:V} to file
        
        
        returns a {(vol_id,layer_id,module_id):(cx,cy,cz)} of superstrips from the first k-most hyperstrips that contain the most tracks   
    """
    
    data = get_mappings()
    hyperstrips_dict = {}
    for key, lists in data.items():
        superstrip_info = []
        hyperstrip_info = [] #hyperstrip is a subset of superstrips for a given track
        for small_dictionaries in lists:
            if 'hit_id' in small_dictionaries:
                del small_dictionaries['hit_id']
            superstrip = (small_dictionaries['volume_id'], small_dictionaries['layer_id'], small_dictionaries['module_id'])
            hyperstrip_info.append(superstrip)

        hyperstrip = tuple(hyperstrip_info)
        #print(hyperstrip)
        if hyperstrip in hyperstrips_dict:
            #print('found one!')
            hyperstrips_dict[hyperstrip] += 1
        else:
            hyperstrips_dict[hyperstrip] = 1

    sorted_hyperstrips = sorted(hyperstrips_dict.items(), key=lambda kv: kv[1], reverse=True)

    for idx, items in enumerate(sorted_hyperstrips):
        hyperstrip, count = items
        if len(hyperstrip) > 1 :
            start = idx
            #print(sorted_hyperstrips[start: start + 500])
            someslice = sorted_hyperstrips[start:start+k]
            break

    #creates a dicitonary for frequency of hits in hyperstrips
    lengths_freq = {}
    for elem in someslice:
        hyperstrip, number_of_tracks = elem 
        hyperstrip_len = len(hyperstrip)
        if hyperstrip_len in lengths_freq:
            lengths_freq[hyperstrip_len] += 1
        else:
            lengths_freq[hyperstrip_len] = 1

    detector_info_path = 'data/detectors.csv'
    detectors = pd.read_csv(detector_info_path)

    superstrip_locs = {}
    json_data = {}
    for hyperstrip in someslice:
        subset_of_strips, tracks = hyperstrip
        for superstrip in subset_of_strips:
            vol, lay, mod = superstrip
            x = detectors.loc[(detectors['volume_id'] == vol) & (detectors['layer_id'] == lay) & (detectors['module_id'] == mod)]['cx'].item()
            y = detectors.loc[(detectors['volume_id'] == vol) & (detectors['layer_id'] == lay) & (detectors['module_id'] == mod)]['cy'].item()
            z = detectors.loc[(detectors['volume_id'] == vol) & (detectors['layer_id'] == lay) & (detectors['module_id'] == mod)]['cz'].item()
            superstrip_locs[superstrip] = (x,y,z) 
            json_data[str(superstrip)] = (x,y,z)
    if toFile:
        with open('top-hyperstrips.json','w') as outfile:
            json.dump(json_data, outfile)
    return superstrip_locs

    


In [12]:
def closestHit(x, y, z, hits):
    hitList = hits.as_matrix(columns=hits.columns[0:4])
    minDist = np.sqrt((x - hitList[0][1])**2 + (y - hitList[0][2])**2 + (z - hitList[0][3])**2)
    for index, hit in enumerate(hitList):
        dist = np.sqrt((x - hit[1])**2 + (y - hit[2])**2 + (z - hit[3])**2)
        if dist < minDist:
            minDist = dist
            minDistHitID = hit[0]
    return minDistHitID



def calc_dist(x,y,z, hits_array):
    min_dist = np.Inf
    best_id = 1
    dist_squared = (hits_array[:,1]-x)**2 + (hits_array[:,2]-y)**2 + (hits_array[:,3]-z)**2
    index = np.argmin (dist_squared)
    return (hits_array[index,0], dist_squared[index])
    for hit in hits_array:
        dist_squared = (hit[1]-x)**2 + (hit[2]-y)**2 + (hit[3]-z)**2
        if (dist_squared < min_dist):
            min_dist = dist_squared
            best_id = hit[0]
    return best_id


"\n    Xs, Ys, Zs = hits_df['x'].values, hits_df['y'].values, hits_df['z'].values\n    #Rs, Phis, _ = cartesian_to_3d_polar(Xs, Ys, Zs)\n    #hits_df['r'], hits_df['phi'] = Rs, Phis\n    \n    \n    #test_ref_r = hits_df.loc[(hits_df['hit_id'] == ref_hit_id)]['r']\n    #print(type(test_ref_r))\n    \n    \n    #print('ref_r shape: '+str(ref_r.shape))\n    dist = np.sqrt((Xs - x)**2 + (Ys - y)**2 + (Zs - z)**2)\n    hits_df['dist'] = dist\n    current_min = hits_df.iloc[0].dist\n    best_id = hits_df.iloc[0].hit_id\n    for row in hits_df.itertuples():\n        dist = row.__getattribute__('dist')\n        #print(dist)\n        hit_id = row.__getattribute__('hit_id')\n        if current_min > dist:\n            current_min = dist\n            best_id = hit_id\n        \n    return best_id\n"

In [7]:
def cartesian_to_3d_polar(x, y, z):
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    #s  = np.sin(phi)
    #c  = np.cos(phi)
    return r, phi, z

def polar_to_cartesian(r, phi, z):
    x = r*np.cos(phi)
    y = r*np.sin(phi)
    return [x, y, z]

In [8]:
"""
    hits: tensor of samples x timesteps x features
    bounds: K:V of x_max, x_min, y_max, y_min, z_max, z_min
"""

"""
assuming input is samples x [x_val,y_val,z_val]
"""
def calc_norm(xyz, x_min, x_max, y_min, y_max, z_min, z_max):
    norm_x = ((xyz[0] - x_min)/(x_max - x_min))
    norm_y = ((xyz[1] - y_min)/(y_max - y_min))
    norm_z = ((xyz[2] - z_min)/(z_max - z_min))
    return [norm_x,norm_y,norm_z]

def norm_hits(hits, bounds):
    assert ('x_max' and 'x_min' and 'y_max' and 'y_min' and 'z_max' and 'z_min' in bounds) or ('r_max' and 'r_min' and 'phi_max' and 'phi_min' and 'z_max' and 'z_min' in bounds)
    return np.apply_along_axis(calc_norm, 1, hits, x_min = bounds['x_min'], x_max = bounds['x_max'], y_min = bounds['y_min'], y_max = bounds['y_max'], z_min = bounds['z_min'], z_max = bounds['z_max']) 


"""
input: hits and truth dfs

output: df of particle_id as index and list of particles 
"""

def gen_tracks(truth_df):
    assert(isinstance(truth_df, pd.DataFrame))
    #print(truth_df)
    truth_df['dist'] = np.sqrt(truth_df['tx']**2+truth_df['ty']**2+truth_df['tz']**2)
    
    grouped = truth_df.groupby('particle_id')['hit_id','dist']
    
    a = grouped.apply(lambda x: x.sort_values( by=['dist'],ascending=True))
    final = a.groupby('particle_id')['hit_id'].apply(np.array)

    return final 

##there's still a problem with how to deal with hits that have a particle id of 0

def batch_iter(truth_df, batch_size):
    tracks = gen_tracks(truth_df).values
    np.random.shuffle(tracks) 
    remainder = len(tracks) % batch_size if len(tracks) % batch_size is not 0 else 0
    if remainder is not 0:
        modded_tracks = tracks[:-remainder]
    else:
        modded_tracks = tracks 
    assert(len(modded_tracks)%batch_size is 0)
    for batch in modded_tracks.reshape(-1,batch_size,1):
        yield batch

def get_data(max_seq_len, batch_size, feature_len, truth_df, hits_df, simple = False, full_tracks = False, ideal = False):
    hits = hits_df
    max_seq_len = max_seq_len
    if ideal is True:
        full_tracks = True
        simple = True
    if full_tracks is True:
        max_seq_len = 10
        simple = True
    b_size = batch_size
    features = feature_len #xyz or phi r z
    if simple is True:
        all_data = list(simple_batch_iter(hits_df, truth_df, batch_size, full_tracks, ideal))
    else:
        all_data = list(batch_iter(truth_df,b_size))
    
    #print(all_data)
    for result in all_data:
        batch = []
        batch_lv = []
        labels_tensor = []
        labels_tensor_lv = []
        for track_list in result:
            for hit_id in track_list:
                hit_coord = []
                track = []
                track_lb = []
                lv_pair = []
                lv_tensor = []
                label_coord = []
                for elem in hit_id:
                    x, y, z, layer_id, volume_id = hits.loc[hits['hit_id']== elem]['x'].item(), hits.loc[hits['hit_id']== elem]['y'].item(), hits.loc[hits['hit_id']== elem]['z'].item(), hits.loc[hits['hit_id'] == elem]['volume_id'].item(), hits.loc[hits['hit_id'] == elem]['layer_id'].item()
                    r,phi,z = cartesian_to_3d_polar(x,y,z)
                    hit_coord = [r,phi,z,layer_id, volume_id]
                    label_coord = [r,phi,z]
                    track.append(hit_coord)
                    track_lb.append(label_coord)
                    layer, volume = hits.loc[hits['hit_id'] == elem]['volume_id'].item(), hits.loc[hits['hit_id'] == elem]['layer_id'].item()
                    lv_pair = [layer, volume]
                    lv_tensor.append(lv_pair)
                zeros_to_add = max_seq_len - len(track)
                if zeros_to_add > 0:
                    add_array = np.zeros((zeros_to_add,feature_len))
                    add_array_lb = np.zeros((zeros_to_add, 3)) #3 is hardcoded for xyz/rphiz
                    add_array_lv = np.zeros((zeros_to_add, 2))
                    np_data = np.array(track)
                    np_data_lb = np.array(track_lb)
                    np_data_lv = np.array(lv_tensor)
                    padded_track_data  = np.append(np_data,add_array,axis=0)
                
                    padded_track_data_lb = np.append(np_data_lb, add_array_lb, axis=0)
                    padded_track_data_lv = np.append(np_data_lv, add_array_lv, axis=0)
                elif zeros_to_add < 0:
                    modded_track = track[:zeros_to_add]
                    modded_track_lv = lv_tensor[:zeros_to_add]
                    modded_track_lb = track_lb[:zeros_to_add]
                    padded_track_data_lb = np.array(modded_track_lb)
                    padded_track_data = np.array(modded_track)
                    padded_track_data_lv = np.array(modded_track_lv)
                else:
                    padded_track_data_lb = np.array(track_lb)
                    padded_track_data = np.array(track)
                    padded_track_data_lv = np.array(lv_tensor)
            
            row_label = padded_track_data_lb[1:]
            row_label_lv = padded_track_data_lv[1:]
            padded_row_label = np.append(row_label, np.zeros((1,3)), axis=0) #hardcoded 3 for xyz/rphiz
            padded_row_label_lv = np.append(row_label_lv, np.zeros((1,2)), axis=0)
            labels_tensor.append(padded_row_label)
            labels_tensor_lv.append(padded_row_label_lv)
            batch.append(padded_track_data)
            batch_lv.append(padded_track_data_lv)
            
        padded_batch_data = np.array(batch)
        padded_batch_data_lv = np.array(batch_lv)
        padded_labels = np.array(labels_tensor)
        padded_labels_lv = np.array(labels_tensor_lv)
        #print(padded_labels)
        #print(padded_labels_lv)
        yield padded_batch_data, padded_labels, padded_batch_data_lv, padded_labels_lv
        
def next_batch(max_seq_len, batch_size, feature_len, simple = False, full_tracks = False, ideal = False, full_data = False):
    if full_data:
        path = '/mnt/hdd/trackml_full_data/all_train/'
        all_data = load_dataset(path,parts=['hits','truth'])
    else:    
        all_data = load_dataset('data/train_sample/', parts=['hits','truth'])
    for data in all_data:
        hit_df, truth_df = data[1], data[2]
        yield from get_data(max_seq_len, batch_size, feature_len, truth_df, hit_df, simple, full_tracks, ideal)
        
def get_hit_info(x, y, z): #returns layer, volume, and hit_id
    row = hits.loc[(hits['x'] == x) & (hits['y'] == y) & (hits['z'] == z)]
    return (int(row.iloc[0][4]), int(row.iloc[0][5]), int(row.iloc[0][0]))

def next_seed(hits_from_seeds, batch_size=1):
    for seed in hits_from_seeds:
        track_seed = seed.reshape(1,3,5)
        yield track_seed.tolist()
        
def distance_between_two_points(p1, p2):
    distance = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + (p2[2] - p1[2])**2)
    return distance

def get_xyz(hit_id):
    xyz = hits[hits["hit_id"] == hit_id].iloc[:,1:4].values.tolist()[0]
    return xyz

## Cylinder hit related methods

In [9]:


def is_cylinder (volume_id, layer_id):
    return (((volume_id == 8 or volume_id == 13) and layer_id in [2,4,6,8])
           or (volume_id == 17 and layer_id in [2,4]))


def layer_number (volume_id, layer_id):
    assert (volume_id in [8, 13, 17]), "This volume_id is not in a cylinder!"
    assert (((volume_id == 8 or volume_id == 13) and layer_id in [2,4,6,8])
           or (volume_id == 17 and layer_id in [2,4])), "This layer_id is not in this volume_id"
    diction = {(8,2):1, (8,4):2, (8,6):3, (8,8):4, (13,2):5, (13,4):6, (13,6):7, (13,8):8, (17,2):9, (17,4):10}
    return diction[(volume_id, layer_id)]

"""
returns dataframe of hits that only lie on cylinders for a given event
"""
def get_cylinder_hits(hit_dataframe, truth_dataframe ):
    
    volume_layer_list = [(8,2),(8,4),(8,6),(8,8),(13,2),(13,4),(13,6),(13,8),(17,2),(17,4)]
    temp = [] 
    merge_by_hit_ids = hit_dataframe.merge(truth_dataframe, how = 'inner', on ='hit_id')
    for pair in volume_layer_list:
        volume, layer = pair
        test_data = merge_by_hit_ids[(merge_by_hit_ids['volume_id'].values == volume) & (merge_by_hit_ids['layer_id'].values == layer)]
        temp.append(test_data)
    return pd.concat(temp)
    
#set full_tracks flag if looking for tracks of len 10
#set ideal flag if looking for tracks of len 10 and only 1 hit per layer
#setting ideal flag will set/override full_tracks flag

def gen_simple_tracks(hit_df, truth_df, full_tracks = False, ideal = False, debug=False):
    if ideal is True:
        full_tracks = True
    assert(isinstance(hit_df, pd.DataFrame) and isinstance(truth_df, pd.DataFrame))
    hit_and_truth = get_cylinder_hits(hit_df, truth_df)
    prelim_tracks = gen_tracks(hit_and_truth).values
    if full_tracks is False:
        return prelim_tracks 
    
    #full_tracks flag must be true at this point
    rows_to_remove = []
    for row, track in enumerate(prelim_tracks):
        if len(track) is not 10: #10 for each vol-layer in detector
            rows_to_remove.append(row)
    full_tracks = np.delete(prelim_tracks, rows_to_remove)
    if ideal is False:
        if debug:
            print('prelim track num: '+str(len(prelim_tracks)))
            print('removed tracks: '+str(len(rows_to_remove)))
            print('remaining tracks: '+str(len(prelim_tracks) - len(rows_to_remove)))
        return full_tracks
    
    #ideal flag must be true at this point
    bad_rows = []
    compare = [x for x in range(1,11)] #to guarantee each hit in track per layer
    for idx, track in enumerate(full_tracks):
        layer_nums = []
        for hit in track:
            vol_id = hit_df.loc[hit_df['hit_id']==hit]['volume_id'].item()
            lay_id = hit_df.loc[hit_df['hit_id']==hit]['layer_id'].item()
            layer_nums.append(layer_number(vol_id, lay_id))
        if layer_nums != compare:
            bad_rows.append(idx)
    ideal_tracks = np.delete(full_tracks, bad_rows)
    if debug:
        print('full tracks: '+str(len(full_tracks)))
        print('non-ideal tracks: '+str(len(bad_rows)))
        print('ideal tracks: '+str(len(ideal_tracks)))
    return ideal_tracks 
            
def simple_batch_iter(hit_df, truth_df, batch_size, full_tracks = False, ideal = False):
    if ideal is True:
        full_tracks = True
    tracks = gen_simple_tracks(hit_df, truth_df, full_tracks, ideal)
    np.random.shuffle(tracks)
    remainder = len(tracks) % batch_size if len(tracks) % batch_size is not 0 else 0
    if remainder is not 0:
        modded_tracks = tracks[:-remainder]
    else:
        modded_tracks = tracks 
    assert(len(modded_tracks)%batch_size is 0)
    for batch in modded_tracks.reshape(-1,batch_size,1):
        yield batch
        
        

## Seeding functions

In [10]:
"""
constants needed for seeding
"""
#hit_id - 1 is the index of the hit
hit_idi = 0
ri = 1
phii = 2
zi = 3
volume_idi = 4
layer_idi = 5

d_phi = 3*np.pi/180
d_phi2 = 3*np.pi/180
d_z = 5
d_r = 5

z_max = 20#maximum distance along z axis of track origin

r_first_layer = 32.313498434074106
r_second_layer = 72.14554366164577
r_third_layer = 116.08917912535651
z_first_disk_right = 599.9792565549286
z_first_disk_left = -z_first_disk_right
z_second_disk_right = 699.9354521625164
z_second_disk_left = -z_second_disk_right


def trans_to_cylindrical (hits):
    hits_trans_info = []
    for row in hits.itertuples():
        hit_id = row.__getattribute__('hit_id')
        x = row.__getattribute__('x')
        y = row.__getattribute__('y')
        z = row.__getattribute__('z')
        volume_id = row.__getattribute__('volume_id')
        layer_id = row.__getattribute__('layer_id')
        r, phi, z = cartesian_to_3d_polar(x,y,z)
        hits_trans_info.append([hit_id, r, phi, z, volume_id, layer_id])
    hits_trans = pd.DataFrame (hits_trans_info, columns=["hit_id", "r", "phi", "z", "volume_id", "layer_id"])
    return hits_trans


def quadrant_shift (phi):
    if (phi < 0):
        return 2*np.pi + phi
    else:
        return phi

    
def extrapolate (x1, x2, x3, y1, y2):
    #assert (x2 - x1 != 0)
    m = (y2 - y1)/(x2 - x1)
    b = y1 - x1*m
    y3 = x3*m + b
    return y3


def create_seeds (hits):#already transformed to cylindrical
    
    hits_trans = trans_to_cylindrical (hits)
    hits_array = np.array (hits_trans.values)
    
    hits_first_layer = hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 2)]
    hits_first_layer_array = hits_first_layer.get_values()
    hits_second_layer = hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 4)]
    hits_second_layer_array = hits_second_layer.get_values()
    hits_third_layer = hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 6)]
    hits_third_layer_array = hits_third_layer.get_values()
    hits_first_disk_left = hits_trans.loc[(hits_trans['volume_id'] == 7) & (hits_trans['layer_id'] == 14)]
    hits_first_disk_left_array = hits_first_disk_left.get_values()
    hits_first_disk_right = hits_trans.loc[(hits_trans['volume_id'] == 9) & (hits_trans['layer_id'] == 2)]
    hits_first_disk_right_array = hits_first_disk_right.get_values()
    hits_second_disk_left = hits_trans.loc[(hits_trans['volume_id'] == 7) & (hits_trans['layer_id'] == 12)]
    hits_second_disk_left_array = hits_second_disk_left.get_values()
    hits_second_disk_right = hits_trans.loc[(hits_trans['volume_id'] == 9) & (hits_trans['layer_id'] == 4)]
    hits_second_disk_right_array = hits_second_disk_right.get_values()
    
    seeds = []
    pool = Pool(32)
    parameters = []
    for hit in hits_first_layer_array:
        parameters.append((hits_array, hits_second_layer_array, hits_third_layer_array, hits_first_disk_left_array, hits_first_disk_right_array, hits_second_disk_left_array, hits_second_disk_right_array, int(np.round(hit[hit_idi]))))
    seeds = pool.starmap (create_seeds_from_first_hit, parameters)
    #for parameter in parameters:
        #create_seeds_from_first_hit (parameter[0], parameter[1], parameter[2], parameter[3], parameter[4], parameter[5], parameter[6], parameter[7])
    flattened_seeds = []
    for some_seeds in seeds:
        for seed in some_seeds:
            flattened_seeds.append (seed)
    return flattened_seeds


def create_seeds_from_first_hit (hits_array, hits_second_layer_array, hits_third_layer_array, hits_first_disk_left_array, hits_first_disk_right_array, hits_second_disk_left_array, hits_second_disk_right_array, hit_id):
    seeds = []
    hit = hits_array[hit_id - 1]
    assert (hit[layer_idi] == 2)
    phi = hit[phii]
    z = hit[zi]
    if (abs(phi) > np.pi/2):
        phi = quadrant_shift (phi)
    phiMin = phi - d_phi/2
    phiMax = phi + d_phi/2
    zMin = extrapolate (0, r_first_layer, r_second_layer, z_max, z)
    zMax = extrapolate (0, r_first_layer, r_second_layer, -z_max, z)
    for thisHit in hits_second_layer_array:
        thisPhi = thisHit[phii]
        if (phiMax > np.pi):
            thisPhi = quadrant_shift (thisPhi)
        thisZ = thisHit[zi]
        if (thisPhi < phiMax and thisPhi > phiMin and thisZ < zMax and thisZ > zMin):
            seeds += create_seeds_from_second_hit (hits_array, hits_third_layer_array, (hit_id, int(np.round(thisHit[hit_idi]))))
            #seeds += create_seeds_from_second_hit_first_disk (hits_array, hits_first_disk_left_array, hits_first_disk_right_array, (hit_id, int(np.round(thisHit[hit_idi]))))
    return seeds #uncomment above line and remove this line to start doing disks too
    r = hit[ri]
    if (z > z_max):
        rMax = extrapolate (z_max, z, z_first_disk_right, 0, r)
        rMin = extrapolate (-z_max, z, z_first_disk_right, 0, r)
        for thisHit in hits_first_disk_right_array:
            thisPhi = thisHit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = thisHit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                seeds += create_seeds_from_second_hit_second_disk (hits_array, hits_second_disk_left_array, hits_second_disk_right_array, (hit_id, int(np.round(thisHit[hit_idi]))))
    if (z < -z_max):
        rMin = extrapolate (z_max, z, z_first_disk_left, 0, r)
        rMax = extrapolate (-z_max, z, z_first_disk_left, 0, r)
        for thisHit in hits_first_disk_left_array:
            thisPhi = thisHit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = thisHit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                seeds += create_seeds_from_second_hit_second_disk (hits_array, hits_second_disk_left_array, hits_second_disk_right_array, (hit_id, int(np.round(thisHit[hit_idi]))))
    return seeds


def create_seeds_from_second_hit (hits_array, hits_third_layer_array, twoSeed):
    seeds = []
    phi1 = hits_array[twoSeed[0] - 1][phii]
    phi2 = hits_array[twoSeed[1] - 1][phii]
    r1 = hits_array[twoSeed[0] - 1][ri]
    r2 = hits_array[twoSeed[1] - 1][ri]
    r3 = r_third_layer
    z1 = hits_array[twoSeed[0] - 1][zi]
    z2 = hits_array[twoSeed[1] - 1][zi]
    phi3 = extrapolate (r1, r2, r3, phi1, phi2)
    z3 = extrapolate (r1, r2, r3, z1, z2)
    if (abs(phi3) > np.pi/2):
        phi3 = quadrant_shift (phi3)
    phiMin = phi3 - d_phi2/2
    phiMax = phi3 + d_phi2/2
    zMin = z3 - d_z/2
    zMax = z3 + d_z/2
    for hit in hits_third_layer_array:
        thisPhi = hit[phii]
        if (phiMax > np.pi):
            thisPhi = quadrant_shift (thisPhi)
        thisZ = hit[zi]
        if (thisPhi < phiMax and thisPhi > phiMin and thisZ < zMax and thisZ > zMin):
            seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    return seeds


def create_seeds_from_second_hit_first_disk (hits_array, hits_first_disk_left_array, hits_first_disk_right_array, twoSeed):
    seeds = []
    phi1 = hits_array[twoSeed[0] - 1][phii]
    phi2 = hits_array[twoSeed[1] - 1][phii]
    r1 = hits_array[twoSeed[0] - 1][ri]
    r2 = hits_array[twoSeed[1] - 1][ri]
    z1 = hits_array[twoSeed[0] - 1][zi]
    z2 = hits_array[twoSeed[1] - 1][zi]
    if (z2 > z1):
        z3 = z_first_disk_right
    else:
        z3 = z_first_disk_left
    phi3 = extrapolate (z1, z2, z3, phi1, phi2)
    r3 = extrapolate (z1, z2, z3, r1, r2)
    if (abs(phi3) > np.pi/2):
        phi3 = quadrant_shift (phi3)
    phiMin = phi3 - d_phi2/2
    phiMax = phi3 + d_phi2/2
    rMin = r3 - d_r/2
    rMax = r3 + d_r/2
    if (z2 > z1):
        for hit in hits_first_disk_right_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    else:
        for hit in hits_first_disk_left_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    return seeds


def create_seeds_from_second_hit_second_disk (hits_array, hits_second_disk_left_array, hits_second_disk_right_array, twoSeed):
    seeds = []
    phi1 = hits_array[twoSeed[0] - 1][phii]
    phi2 = hits_array[twoSeed[1] - 1][phii]
    r1 = hits_array[twoSeed[0] - 1][ri]
    r2 = hits_array[twoSeed[1] - 1][ri]
    z1 = hits_array[twoSeed[0] - 1][zi]
    z2 = hits_array[twoSeed[1] - 1][zi]
    if (z2 > z1):
        z3 = z_second_disk_right
    else:
        z3 = z_second_disk_left
    phi3 = extrapolate (z1, z2, z3, phi1, phi2)
    r3 = extrapolate (z1, z2, z3, r1, r2)
    if (abs(phi3) > np.pi/2):
        phi3 = quadrant_shift (phi3)
    phiMin = phi3 - d_phi2/2
    phiMax = phi3 + d_phi2/2
    rMin = r3 - d_r/2
    rMax = r3 + d_r/2
    if (z2 > z1):
        for hit in hits_second_disk_right_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    else:
        for hit in hits_second_disk_left_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    return seeds

def rpzlv_from_seeds (seeds, hits):
    hits_from_seeds = []
    for seed in seeds:
        hits_from_seed = []
        for hit_id in seed:
            hit = hits.loc[hits['hit_id'] == hit_id]
            x = hit['x'].values[0]
            y = hit['y'].values[0]
            z = hit['z'].values[0]
            r, phi, z = cartesian_to_3d_polar (x, y, z)
            l = hit['layer_id'].values[0]
            v = hit['volume_id'].values[0]
            hits_from_seed.append ([r, phi, z, l, v])
        hits_from_seeds.append (hits_from_seed)
    return np.array (hits_from_seeds)